# Characterizing Patronage on YouTube

## 0. Files and brief explanation of those

All data is located in `/dlabdata1/youtube_large/`

**YouNiverse dataset:**

- `df_channels_en.tsv.gz`: channel metadata.
- `df_timeseries_en.tsv.gz`: channel-level time-series.
- `yt_metadata_en.jsonl.gz`: raw video metadata.
- `youtube_comments.tsv.gz`: user-comment matrices.
- `youtube_comments.ndjson.zst`: raw comments — this is a HUGE file.

**Graphteon dataset:**
- `creators.csv` list with all creator names.
- `final_processed_file.jsonl.gz` all graphteon time-series.
- `pages.zip` raw html of the pages in graphteon.

In [ ]:
# !conda list

In [ ]:
import os 
import io
import pandas as pd
import json
import re
import tqdm
import zstandard
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from tqdm import tqdm

In [ ]:
DATA_FOLDER = "/dlabdata1/youtube_large/"

In [ ]:
# list all files in DATA_FOLDER
# !ls -lh /dlabdata1/youtube_large

## 1. Load data

### 1.1. YouNiverse dataset

#### 1.1.1 Channel metadata
Metadata associated with the 136,470 channels: **channel ID**, **join date**, **country**, **number of subscribers**, **most frequent category**, and the **channel’s position** in socialblade.com’s subscriber ranking. \
The number of subscribers is provided both as obtained from channelcrawler.com (between 2019-09-12 and 2019-09-17) and as crawled from socialblade.com (2019-09-27). Additionally, we also provide a set of **weights** (derived from socialblade.com’s subscriber rankings) that can be used to partially correct sample biases in our dataset.

- `category_cc`: most frequent category ??
- `join_date`
- `channel`
- `name_cc`
- `subscribers_cc`: number of subscribers
- `videos_cc`
- `subscriber_rank_sb`: channel’s position in socialblade.com’s subscriber ranking
- `weights`: Set of derived from socialblade.com’s subscriber rankings. Can be used to partially correct sample biases in our dataset.

In [ ]:
# !ls -lh /dlabdata1/youtube_large/df_channels_en.tsv.gz

In [ ]:
# channel metadata
df_yt_channels = pd.read_csv(DATA_FOLDER+'df_channels_en.tsv.gz', sep="\t", compression='gzip')
df_yt_channels

##### 1.1.1.1 Summary statistics

In [ ]:
print('Number of unique categories:         {:,}'.format(df_yt_channels['category_cc'].nunique()))
print('Number of unique channels:      {:,}'.format(df_yt_channels['channel'].nunique()))
print('Number of unique channel names: {:,}'.format(df_yt_channels['name_cc'].nunique()))

print('\nNote: there are more unique channels than unique names, so some channels might have the same name!')

##### 1.1.1.2 Distribution of videos and subscribers

In [ ]:
selected_cols = ['videos_cc', 'subscribers_cc']

# plot with linear scale for x axis 
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10,5))

for i,(col,ax) in enumerate(zip(selected_cols, axs.flatten())):
    sns.histplot(data=df_yt_channels[col], ax=ax, bins=50, kde=False, color=f'C{i}')
    ax.set(title=f'Distribution of {col}')
    ax.set_ylabel("Count - number of channels")
    ax.set(yscale="log")
plt.tight_layout()
plt.show()

# plot with log scale for x axis 
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10,5))

for i,(col,ax) in enumerate(zip(selected_cols, axs.flatten())):
    sns.histplot(data=df_yt_channels[col], ax=ax, bins=2000, kde=False, color=f'C{i}')

    ax.set(title=f'Distribution of {col} (log-log scale)')
    ax.set_ylabel("Count - number of channels")
    ax.set(yscale="log")
    ax.set(xscale="log")
plt.tight_layout()
plt.show()

# descriptive statistics table
df_yt_channels[selected_cols].describe().T

**Discussion:** \
From the above graphs and table, we can see that _videos_ and _subscribers_ distributions among YouTube channels follow a **power law**, meaning that most channels have a only a few videos and a few subscribers, but a few of them have a lot of videos and a lot of subscribers.

More specifically:
- 50% of the YouTube channels have less than 175 videos
- 50% of the YouTube channels have less than 42,400 subscribers videos

_Note: only channels with at least 10 videos and 10,000 subscribers were considered for this study._

#### 1.1.2 Channel time-series
(Weekly number of viewers and subscribers)

Time series of channel activity at **weekly granularity**. The span of time series varies by channel depending on when socialblade.com started tracking the channel. On average, it contains **2.8 years of data per channel** for **133k channels** (notice that this means there are roughly 4k channels for which there is no time-series data). \
Each data point includes the **number of views** (`views`) and **subscribers** (`subs`) obtained in the given week, as well as the **number of videos** (`videos`) posted by the **channel** (`channel`). The number of videos is calculated using the video upload dates in our video metadata, such that videos that were unavailable at crawl time are not accounted for. 

---

Time series related to each channel.\
These come from a mix of YouTube data and time series crawled from [socialblade.com](https://socialblade.com/):
- From the former (YouTube data): derived weekly time series indicating **how many videos each channel had posted per week**. 
- From the latter (socialblade.com): crawled weekly statistics on the **number of viewers** `views` and **subscribers** `subs` per channel `channel`. This data was available for around 153k channels.

    - `channel`: unique channel ID, which is the numbers and letters at the end of the URL.
    - `category`: Content categories organize channels and videos on YouTube and help creators, advertisers, and channel managers identify with content and audiences they wish to associate with.
    - `datetime`: date of first day of the week?
    - `views`: number of views obtained in the given week
    - `delta_views`: difference of number of views between current and former week 
    - `subs`: number of subscribers per channel
    - `delta_subs`: difference of number of subscribers between current and former week
    - `videos`: number of videos posted by the channel up to date
    - `delta_videos`: difference of number of videos posted by the channel between current and former week
    - `activity`: ???
    
    
Note: Can view the channel by appending the channel id to the url, e.g.  https://www.youtube.com/channel/UCBJuEqXfXTdcPSbGO9qqn1g


In [ ]:
# !ls -lh /dlabdata1/youtube_large/df_timeseries_en.tsv.gz

In [ ]:
# channel-level time-series.
df_yt_timeseries = pd.read_csv(DATA_FOLDER+'df_timeseries_en.tsv.gz', sep="\t", compression='gzip', parse_dates=['datetime'])
df_yt_timeseries

##### 1.1.2.1 Summary statistics

In [ ]:
# df_yt_timeseries.describe().T

In [ ]:
print('Timeseries data was gathered between {} and {}'.format(df_yt_timeseries['datetime'].min().strftime('%B %d, %Y'),
                                                         df_yt_timeseries['datetime'].max().strftime('%B %d, %Y')))
print('Total number of datapoints accross all channels: {:>12,}'.format(len(df_yt_timeseries)))
data_points_dist = df_yt_timeseries['channel'].value_counts()
print('Average number of datapoints per channel:       {:>12.0f} weeks (≈{:,.1f} years)'.format(data_points_dist.mean(), data_points_dist.mean()/52))
print('Number of unique categories:                     {:>12,}'.format(df_yt_timeseries['category'].nunique()))
print('Number of unique channels:                       {:>12,}'.format(df_yt_timeseries['channel'].nunique()))

##### Ditribution of datapoints per year

In [ ]:
yt_ts_year_cnt = df_yt_timeseries.groupby(df_yt_timeseries.datetime.dt.year).size()

In [ ]:
print('Timeseries data was gathered between {} and {}'.format(df_yt_timeseries['datetime'].min().strftime('%B %d, %Y'),
                                                         df_yt_timeseries['datetime'].max().strftime('%B %d, %Y')))
yt_ts_year_cnt.plot(kind='bar')
plt.title("Nb of datapoints per year accross all channels")
plt.xlabel("Year")
plt.ylabel("Count (datapoints)")
plt.show()

data_per_cat['subscribers_cc_sum'].sort_values(ascending=False)

yt_ts_year_cnt

##### Ditribution of datapoints per month

In [ ]:
yt_ts_month_cnt = df_yt_timeseries_merged.groupby([df_yt_timeseries_merged.datetime.dt.year, df_yt_timeseries_merged.datetime.dt.month]).size()

In [ ]:
# plot number of datapoints per month
plt.figure(figsize=(15,6))
yt_ts_month_cnt.plot(kind='bar')
plt.title("Number of datapoints per month accross all channels")
plt.xlabel("Month")
plt.ylabel("Count (datapoints)")
plt.show()

yt_ts_month_cnt.head(24)

##### Distribution of datapoints accross channels

In [ ]:
# Distribution of datapoints accross channels

print('Total number of datapoints accross all channels: {:>12,}'.format(len(df_yt_timeseries)))
data_points_dist = df_yt_timeseries['channel'].value_counts()
print('Average number of datapoints per channel:        {:>12,.0f} weeks (≈{:,.1f} years)'.format(data_points_dist.mean(), data_points_dist.mean()/52))

ax = sns.histplot(data=data_points_dist, bins=50, kde=False, color=f'C{1}')

ax.set(title=f'Distribution of datapoints (weeks) accross channels')
ax.set_xlabel('number of data points (weeks)')
ax.set_ylabel('number of channels')

# ax.set(yscale="log")
# plt.tight_layout()
plt.show()

In [ ]:
sel_cols = ['datetime', 'views', 'delta_views', 'subs', 'delta_subs', 'videos', 'delta_videos', 'activity']
data_per_channel = df_yt_timeseries.groupby('channel')[sel_cols].agg(['min', 'max', 'count', 'mean'])
data_per_channel.head()

#####  1.1.2.2 Views per channel

In [ ]:
data_per_channel['views'].head()

In [ ]:
# Distribution of total views per channel

fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(6,8))

# print(axs[0])
sns.histplot(data=data_per_channel['views']['max'], ax=axs[0], bins=20, kde=False, color=f'C{1}')

axs[0].set(title=f'Distribution of total views per channel')
axs[0].set_xlabel('number of views (in billions)')
axs[0].set_ylabel('number of channels')

axs[0].set(yscale="log")

xlabels1 = ['{:,.0f}'.format(x) + 'bn' for x in axs[0].get_xticks()/1_000_000_000]
axs[0].set_xticklabels(xlabels1)


# # Distribution of total views per channel (log scale)
sns.histplot(data=data_per_channel['views']['max'], ax=axs[1], bins=1000, kde=False, color=f'C{1}')

axs[1].set(title=f'Distribution of total views per channel (log-log scale)')
axs[1].set_xlabel('number of views (in millions)')
axs[1].set_ylabel('number of channels')

axs[1].set(yscale="log")
axs[1].set(xscale="log")

# plt.ticklabel_format(style='plain', axis='x')

xlabels2 = ['{:,.0f}'.format(x) + 'M' for x in axs[1].get_xticks()/1_000_000]
axs[1].set_xticklabels(xlabels2)

plt.tight_layout()
plt.show()

data_per_channel['views'][['max']].describe().T

In [ ]:
print("Top 10 channels with the most total views (in billions):")

for index, value in data_per_channel['views']['max'].sort_values(ascending=False)[:10].items():
    print('https://www.youtube.com/channel/{} : {:,.1f} bn views'.format(index, value/1_000_000_000))

##### 1.1.2.3. Videos per channel

In [ ]:
data_per_channel['videos'].head()

In [ ]:
# Distribution of total videos per channel
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(6,8))
sns.histplot(data=data_per_channel['videos']['max'], ax=axs[0], bins=20, kde=False, color=f'C{1}')

axs[0].set(title=f'Distribution of total videos per channel')
axs[0].set_xlabel('number of videos')
axs[0].set_ylabel('number of channels')
axs[0].set(yscale="log")

# # Distribution of total views per channel (log scale)
sns.histplot(data=data_per_channel['videos']['max'], ax=axs[1], bins=100, kde=False, color=f'C{1}')

axs[1].set(title=f'Distribution of total videos per channel (log-log scale)')
axs[1].set_xlabel('number of videos')
axs[1].set_ylabel('number of channels')
axs[1].set(yscale="log")
axs[1].set(xscale="log")

plt.tight_layout()
plt.show()

data_per_channel['videos'][['max']].describe().T

In [ ]:
print("Top 10 channels with the most total videos:")

for index, value in data_per_channel['videos']['max'].sort_values(ascending=False)[:10].items():
    print('https://www.youtube.com/channel/{} : {:,.0f} videos'.format(index, value))

##### 1.1.2.4. Subscribers per channel

In [ ]:
data_per_channel['subs'].head()

In [ ]:
# Distribution of total videos per channel
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(6,8))
sns.histplot(data=data_per_channel['subs']['max'], ax=axs[0], bins=20, kde=False, color=f'C{1}')

axs[0].set(title=f'Distribution of total subscribers per channel')
axs[0].set_xlabel('number of subscribers (in millions)')
axs[0].set_ylabel('number of channels')
axs[0].set(yscale="log")
xlabels0 = ['{:,.0f}'.format(x) + 'M' for x in axs[0].get_xticks()/1_000_000]
axs[0].set_xticklabels(xlabels0)

# # Distribution of total views per channel (log scale)
sns.histplot(data=data_per_channel['subs']['max'], ax=axs[1], bins=500, kde=False, color=f'C{1}')

axs[1].set(title=f'Distribution of total subscribers per channel (log-log scale)')
axs[1].set_xlabel('number of subscribers')
axs[1].set_ylabel('number of channels')
axs[1].set(yscale="log")
axs[1].set(xscale="log")

plt.tight_layout()
plt.show()

data_per_channel['videos'][['max']].describe().T

In [ ]:
data_per_channel['subs']['max'].sort_values(ascending=False)[:10]

In [ ]:
print("Top 10 channels with the most total subscribers:")

for index, value in data_per_channel['subs']['max'].sort_values(ascending=False)[:10].items():
    print('https://www.youtube.com/channel/{} : {:,.1f}M subscribers'.format(index, value/1_000_000))

In [ ]:
# set the columns to the top level of the multi-index
# data_per_channel.columns = data_per_channel.columns.get_level_values(0)
# data_per_channel

#### 1.1.3 Raw video metadata

In [ ]:
!ls -lh /dlabdata1/youtube_large/yt_metadata_en.jsonl.gz

In [ ]:
# ! zcat /dlabdata1/youtube_large/yt_metadata_en.jsonl.gz | head

In [ ]:
df_yt_metadata = pd.read_json(DATA_FOLDER+'yt_metadata_en.jsonl.gz', compression='gzip', lines=True, nrows=100)
df_yt_metadata.head(2)

#### 1.1.4 user-comment matrices

In [ ]:
# !ls -lh /dlabdata1/youtube_large/youtube_comments.tsv.gz

In [ ]:
# user-comment matrices
df_yt_comments = pd.read_csv(DATA_FOLDER+'youtube_comments.tsv.gz', sep="\t", compression='gzip', nrows=100)
df_yt_comments.head()

#### 1.1.5 raw comments

In [ ]:
# !ls -lh /dlabdata1/youtube_large/youtube_comments.ndjson.zst

In [ ]:
def line_jsonify(line): 
    """

    :param line: string to parse and jsonify
    :return: 
    """    
    
    # add square brackets around line
    line = "[" + line + "]"

    # remove quotes before and after square brackets   
    line = line.replace("\"[{", "[{")
    line = line.replace("}]\"", "}]")    
    
    # replace double double-quotes with single double-quotes
    line = line.replace("{\"\"", "{\"")
    line = line.replace("\"\"}", "\"}")
    line = line.replace("\"\":\"\"", "\":\"")
    line = line.replace(":\"\"", ":\"")
    line = line.replace("\"\":", "\":")
    
    # line = line.replace("\"\":", "\":")
    line = line.replace("\"\",\"\"", "\",\"")
    line = line.replace("\"\",\"\"", "\",\"")
    line = line.replace("\\\"\"", "\\\"")
    line = line.replace("\\\",[", "\\\\ \",[")
    
    line = re.sub(r',\"\"(?!\,)', ',\"', line)

    line = line.replace("true,\"\"", "true,\"")
    line = line.replace("false,\"\"", "false,\"")
    
    return line

In [ ]:
class Zreader:

    def __init__(self, file, chunk_size=16384):
        '''Init method'''
        import codecs
        self.fh = open(file,'rb')
        print(f"reading {file} in chunks ...")
        self.chunk_size = chunk_size
        self.dctx = zstandard.ZstdDecompressor(max_window_size=2147483648)
        self.reader = self.dctx.stream_reader(self.fh)
        self.buffer = ''

    def readlines(self):
        '''Generator method that creates an iterator for each line of JSON'''
        nb_chunk = 0
        while True:
            nb_chunk = nb_chunk + 1
            if nb_chunk % 5000 == 0:
                print("number of chunks read: ", nb_chunk)
                
            chunk = self.reader.read(self.chunk_size).decode("utf-8", "replace")

            if not chunk:
                break
            lines = (self.buffer + chunk).split("\n")

            # print("lines per chunk: ", len(lines))
            # print(lines)
            
            for line in lines[:-1]:
                # print(line)
                yield line

            self.buffer = lines[-1]

In [ ]:
NB_OF_LINES = 350000
lines_json = []
inp_file = DATA_FOLDER+"youtube_comments.ndjson.zst"
reader = Zreader(inp_file, chunk_size=4092)

for i, line in enumerate(reader.readlines()):
    if i > NB_OF_LINES:
        # print(line)
        break
    line_json = json.loads(line_jsonify(line))
    lines_json.append(line_json)

print("==> number of lines read:", len(lines_json))

df_yt_comments_raw = pd.DataFrame(data=lines_json[1:], columns=lines_json[0])
df_yt_comments_raw.head()

### 1.2. Graphtreon dataset

#### 1.2.1 List with all creator names.

In [ ]:
# !ls -lh /dlabdata1/youtube_large/creators.csv

In [ ]:
# list with all creator names.
df_gt_creators = pd.read_csv(DATA_FOLDER+'creators.csv')
df_gt_creators.head()

#### 1.2.2 All graphtreon time-series

In [ ]:
# !ls -lh /dlabdata1/youtube_large/final_processed_file.jsonl.gz

In [ ]:
# final_processed_file.jsonl.gz all graphteon time-series.
df_gt_timeseries = pd.read_json(DATA_FOLDER+'final_processed_file.jsonl.gz', compression='gzip', lines=True, nrows=100)
df_gt_timeseries.head()

#### 1.2.3 Raw html of the pages in graphteon.

In [ ]:
!ls -lh /dlabdata1/youtube_large/pages.zip

In [ ]:
# pages.zip raw html of the pages in graphteon.

## 2. Merge data

### 2.1. Merge channels data with YouTube timeseries

In [ ]:
display(df_yt_timeseries.head(1))
display(df_yt_channels.head(1))

In [ ]:
df_yt_timeseries_merged = df_yt_timeseries.merge(df_yt_channels, how='inner', on='channel')

# remove duplicate columns
# df_yt_timeseries_merged.drop('category_cc', axis='columns', inplace=True)

df_yt_timeseries_merged

### 2.2. Merge YouTube timeseries and Graphtreon timeseries

## 3. EDA

### 3.0 Channel metadata

In [ ]:
df_yt_channels

### 3.1 YouTube time-series EDA
This data is from the YouNiverse dataset, a large collection of channel and video metadata from English-language YouTube. YouNiverse comprises metadata from over 136k channels and 72.9M **videos published between May 2005 and October 2019**, as well as channel-level time-series data with weekly subscriber and view counts.

In [ ]:
# print(f"Total number of datapoints in YouTube timeseries file: {len(df_yt_timeseries_merged):,}")

In [ ]:
# print("Columns of the merged YouTube timeseries dataframe")
# # df_yt_timeseries_merged.columns
# [col for col in df_yt_timeseries_merged.columns]

In [ ]:
df_yt_timeseries_merged.head()

#### Videos per channel

In [ ]:
data_per_channel = df_yt_timeseries_merged.groupby('channel')[['videos_cc', 'subscribers_cc']].agg(['max'])

# set the columns to the top level of the multi-index
data_per_channel.columns = data_per_channel.columns.get_level_values(0)
data_per_channel

In [ ]:
data_per_channel[['videos_cc']].sort_values('videos_cc', ascending=False)

#### Videos and subscribers per channel

In [ ]:
numeric_cols = data_per_channel.columns

# nice plot!
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10,5))

for i,(col,ax) in enumerate(zip(numeric_cols, axs.flatten())):
    sns.histplot(data=data_per_channel[col], ax=ax, bins=50, kde=False, color=f'C{i}')

    ax.set(title=f'Distribution of {col}')
    ax.set(yscale="log")
plt.tight_layout()
plt.show()

# nice table!
data_per_channel.describe().T

In [ ]:
# # the histogram of the data
# plt.hist(data_per_channel['videos_cc'], 100, facecolor='g', log=True, density=True, alpha=0.75)

# # plt.xscale('log',base=10) 

# plt.xlabel('# videos')
# plt.ylabel('Density')
# plt.title('Histogram of videos per channel')
# plt.grid(True)
# plt.show()

In [ ]:
# # MATPLOTLIB EQUIVALENT
# fig, ax = plt.subplots(1,2, figsize=(10,5),  sharey=False, sharex=False)

# # ax[0].ticklabel_format(axis='x', style='scientific', scilimits=(-9,9))
# ax[0].ticklabel_format(axis='x', style='plain')
# ax[0].hist(data_per_channel['videos_cc'], bins = 60, color='blue', log=True, alpha=0.3)
# ax[0].set_title('Videos per channel')
# ax[0].set_xlabel('# videos')    
# ax[0].set_ylabel('count')    
# # ax[0].set_xscale('log', base=10) 


# ax[1].ticklabel_format(axis='x', style='scientific', scilimits=(-6,6))
# ax[1].hist(data_per_channel['subscribers_cc'], bins = 60, color='blue', log=True, alpha=0.3)
# ax[1].set_title('Subscribers per channel')
# ax[1].set_xlabel('# subscribers')    
# ax[1].set_ylabel('count')    
# # ax[1].set_xscale('log', base=10) 

# # fig.suptitle('Distribution of the data', fontweight="bold")
# fig.tight_layout()

#### 3.1.1 Distribution of datetime points per channel

Not all channels timeseries start and end at the same time, therefore we have a different amount of datapoints for each channel

In [ ]:
datetime_data = df_yt_timeseries_merged.groupby('channel')['datetime'].agg(['min', 'max'])
datetime_data.head()

In [ ]:
# datetime_data.describe().T

#### 3.1.1 Distribution per categories

In [ ]:
df_yt_timeseries_merged.head()

In [ ]:
data_per_cat_chan = df_yt_timeseries_merged.groupby(['category', 'channel'])['videos_cc', 'subscribers_cc'].agg(['max'])

# set the columns to the top level of the multi-index
data_per_cat_chan.columns = data_per_cat_chan.columns.get_level_values(0)
data_per_cat_chan

In [ ]:
data_per_cat_chan.reset_index(inplace=True)
data_per_cat_chan

##### Nb of channels per category

In [ ]:
chan_per_cat = data_per_cat_chan.groupby('category')[['channel']].count().sort_values('channel', ascending=False)

In [ ]:
chan_per_cat.plot(kind='bar')
plt.title("Number of channels per category")
plt.xlabel("Categories")
plt.ylabel("Number of channels")
plt.show()
chan_per_cat['channel']

In [ ]:
# data_per_cat = data_per_cat_chan.groupby('category')['videos_cc','subscribers_cc'].agg(['min', 'max', 'count', 'sum'])
data_per_cat = data_per_cat_chan.groupby('category')[['videos_cc','subscribers_cc']].agg(['sum'])
data_per_cat.columns = data_per_cat.columns.get_level_values(0)
data_per_cat = data_per_cat.add_suffix('_sum')
data_per_cat

In [ ]:
print("Number of unique categories: ", len(data_per_cat))

##### Videos per category

In [ ]:
data_per_cat['videos_cc_sum'].sort_values(ascending=False).plot(kind='bar')
plt.title("Number of videos per category")
plt.xlabel("Categories")
plt.ylabel("Number of videos")
plt.show()

data_per_cat['videos_cc_sum'].sort_values(ascending=False)

##### Subscribers per category

In [ ]:
data_per_cat['subscribers_cc_sum'].sort_values(ascending=False).plot(kind='bar')
plt.title("Number of subscribers per category")
plt.xlabel("Categories")
plt.ylabel("Number of videos")
plt.show()

data_per_cat['subscribers_cc_sum'].sort_values(ascending=False)

#### 3.1.2 Distribution of videos per channel

In [ ]:
df_yt_timeseries_merged

In [ ]:
chan_id_dist = df_yt_timeseries_merged[['channel', 'name_cc']].value_counts()
chan_id_dist

In [ ]:
chan_id_dist = df_yt_timeseries_merged['channel'].value_counts()
print("Number of unique channels id are: ", len(chan_id_dist))
print(chan_id_dist)

In [ ]:
chan_id_dist.hist()
plt.title("Histogram: number of data points per channel")
plt.xlabel("Number of data points")
plt.ylabel("Number of channels")
plt.show()

In [ ]:
# some channel names have different ids, ex:
print("Warning: A same channel name can have different ids, for example, the following channel ids all have the same name 'Kai':")
kai_chan_id = df_yt_timeseries_merged[df_yt_timeseries_merged['name_cc'] == 'Kai']['channel'].value_counts()

for id in kai_chan_id.index:
    print('https://www.youtube.com/channel/' + id)

In [ ]:
cat_dist.plot(kind='bar')
plt.title("Distribution of YouTube channels per categories")
plt.xlabel("Categories")
plt.ylabel("Number of videos")
plt.show()

#### 3.1.4 Number of views per category per month

In [ ]:
# df_yt_timeseries_merged